In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# Define a simple feedforward neural network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # Define the layers: input -> hidden -> output
        self.fc1 = nn.Linear(784, 30)  # 784 input features, 30 hidden neurons
        self.fc2 = nn.Linear(30, 10)   # 30 hidden neurons, 10 output neurons (classes)

    def forward(self, x):
        # Pass input through the layers with activation
        x = torch.sigmoid(self.fc1(x))  # Sigmoid activation for hidden layer
        x = torch.sigmoid(self.fc2(x))  # Sigmoid activation for output layer
        return x

In [3]:
# Preprocess the MNIST data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load training and test data
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=10, shuffle=False)


Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [4]:
# Initialize the network, loss function, and optimizer
network = NeuralNetwork()
criterion = nn.MSELoss()  # Mean Squared Error as loss function
optimizer = optim.SGD(network.parameters(), lr=0.5)  # Stochastic Gradient Descent

In [7]:
# Training the network
epochs = 20
for epoch in range(epochs):
    for data in train_loader:
        inputs, labels = data
        inputs = inputs.view(inputs.shape[0], -1)  # Flatten the 28x28 images into 784 input features
        one_hot_labels = torch.zeros(inputs.shape[0], 10).scatter_(1, labels.view(-1, 1), 1.0)  # One-hot encode labels
        
        # Forward pass
        outputs = network(inputs)
        loss = criterion(outputs, one_hot_labels)

        # Backward pass and optimization
        optimizer.zero_grad()  # Reset gradients
        loss.backward()        # Backpropagation
        optimizer.step()       # Update weights

    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

Epoch 1/20, Loss: 0.025630159303545952
Epoch 2/20, Loss: 0.03250505030155182
Epoch 3/20, Loss: 0.01511309389024973
Epoch 4/20, Loss: 0.0166353527456522
Epoch 5/20, Loss: 0.013561785221099854
Epoch 6/20, Loss: 0.013270875439047813
Epoch 7/20, Loss: 0.0009309346205554903
Epoch 8/20, Loss: 0.0007553132600151002
Epoch 9/20, Loss: 0.000743450247682631
Epoch 10/20, Loss: 0.001636252156458795
Epoch 11/20, Loss: 0.002549281809478998
Epoch 12/20, Loss: 0.01818937249481678
Epoch 13/20, Loss: 0.009050672873854637
Epoch 14/20, Loss: 0.0016539384378120303
Epoch 15/20, Loss: 0.0021010988857597113
Epoch 16/20, Loss: 0.003023591125383973
Epoch 17/20, Loss: 0.0003838471311610192
Epoch 18/20, Loss: 0.011204535141587257
Epoch 19/20, Loss: 0.0016658741515129805
Epoch 20/20, Loss: 0.0004991011810488999


In [8]:
# Testing the network
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation for testing
    for data in test_loader:
        inputs, labels = data
        inputs = inputs.view(inputs.shape[0], -1)  # Flatten images
        outputs = network(inputs)
        predicted = torch.argmax(outputs, dim=1)  # Get the index of the highest score (class label)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy}%')

Test Accuracy: 95.04%
